In [1]:
%matplotlib inline

In [2]:
import os
import sys

In [3]:
print sys.version

2.7.14 (default, Jan 17 2018, 15:13:18) 
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)]


In [34]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

In [5]:
from utils import get_xyz_data, get_size

/usr/local/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


In [6]:
from properties import get_element_properties
from properties import ElementExtended

In [7]:
DATA_DIR = './data'

#### Get element properties from files (as check)

In [8]:
elem_properties = get_element_properties(DATA_DIR)

In [9]:
elem_properties

,element,ea,ip,rs_max,rp_max,rd_max
0,In,-0.312500,-5.537400,1.09,1.390000,1.940000
1,Ga,-0.108100,-5.818200,0.99,1.330000,2.160000
2,Al,-0.256300,-5.780000,1.13,1.500000,3.110000
3,O,-0.225633,-5.711867,1.07,1.406667,2.403333


#### Process all files (test and train)

In [35]:
def process_xyz_files(batch='train'):
    """
    Function to process each .xyz file and calculate average properties.
    Returns a dataframe with these quantities for each structure id.
    """
    
    summary_result = []
    
    pbar = tqdm_notebook(os.listdir(os.path.join(DATA_DIR, batch)))
    
    for i in pbar:
        # Load .xyz data
        fname = os.path.join(DATA_DIR, batch, i, 'geometry.xyz')
        pos, latvec, natoms = get_xyz_data(fname)
        
        # Element index
        idx = int(i)
        
        # Properties including O
        avg_mass = pos['atomic_mass'].mean()
        
        # Properties excluding O
        pos = pos[pos['element'] != 'O']
        avg_elec = pos['electroneg'].mean()
        avg_ea = pos['ea'].mean()
        avg_ip = pos['ip'].mean()
        avg_rs_max = pos['rs_max'].mean()
        avg_rp_max = pos['rp_max'].mean()
        avg_rd_max = pos['rd_max'].mean()

        o_cnt = get_size(natoms, 'O')

        row = [idx, avg_elec, avg_mass, o_cnt,
               avg_ea, avg_ip,
               avg_rs_max, avg_rp_max, avg_rd_max]
        
        summary_result.append(row)
    
    summary_result_df = pd.DataFrame(summary_result,
                                     columns=['id', 'avg_elec', 'avg_mass', 'o_cnt',
                                              'avg_ea', 'avg_ip',
                                              'avg_rs_max', 'avg_rp_max', 'avg_rd_max'])
    return summary_result_df.sort_values('id')

#### Process files

In [36]:
data_train = process_xyz_files(batch='train')
data_test = process_xyz_files(batch='test')

HBox(children=(IntProgress(value=0, max=2400), HTML(value=u'')))

HBox(children=(IntProgress(value=0, max=600), HTML(value=u'')))

In [18]:
data_train.head()

,id,avg_elec,avg_mass,o_cnt,avg_ea,avg_ip,avg_rs_max,avg_rp_max,avg_rd_max
1067,1,1.68500,26.803475,48,-0.200725,-5.794325,1.07750,1.436250,2.753750
2327,2,1.68500,26.803475,48,-0.200725,-5.794325,1.07750,1.436250,2.753750
2219,3,1.64750,23.597865,24,-0.228512,-5.787162,1.10375,1.468125,2.931875
2201,4,1.65250,29.175902,18,-0.270350,-5.719350,1.12000,1.472500,2.817500
2342,5,1.79875,44.253090,48,-0.184750,-5.712900,1.02750,1.352500,2.077500


In [19]:
data_test.head()

,id,avg_elec,avg_mass,o_cnt,avg_ea,avg_ip,avg_rs_max,avg_rp_max,avg_rd_max
260,1,1.762188,40.483793,48,-0.206150,-5.714512,1.050625,1.382500,2.262500
578,2,1.660000,24.666402,48,-0.219250,-5.789550,1.095000,1.457500,2.872500
543,3,1.671667,29.097450,18,-0.240967,-5.745933,1.100000,1.453333,2.756667
539,4,1.697500,27.872011,48,-0.191462,-5.796712,1.068750,1.425625,2.694375
583,5,1.763125,39.920105,48,-0.199762,-5.723288,1.047500,1.380625,2.269375


#### Write out extended test and train data files

In [20]:
data_train.to_csv(os.path.join(DATA_DIR, 'train_ext.csv'), index=False)

In [21]:
data_test.to_csv(os.path.join(DATA_DIR, 'test_ext.csv'), index=False)